Have a model go through every word and delete it. Then find the average. If negative add the negative confidence. If positive add the positive confidence. It should average out to a number. It should be between the 0 and then this makes it easier to change the method of change.

In [ ]:
import pandas as pd
from transformers import pipeline
classification_task = pipeline("sentiment-analysis",model = "distilbert-base-uncased-finetuned-sst-2-english")
unmasker = pipeline('fill-mask', model='bert-base-uncased')
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string 
result = string.punctuation 
from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('stsb-roberta-large')
from tabulate import tabulate
import ipysheet
import csv
from timeit import default_timer as timer
import time
import homoglyphs as hg
from itertools import combinations
pyTorchData = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
bertAttackData = pd.read_csv('https://raw.githubusercontent.com/LinyangLee/BERT-Attack/master/data_defense/imdb_1k.tsv', delimiter='\t', header=None)
batch1=pyTorchData[0:5000]
sentences = batch1[0]#here we can easily add the adversarial sentences

In [ ]:
def replace(sentence):
    sentences = []
    for k in sentence:
            if k in result:
                formatted = sentence.replace(k," "+k+" ")
            else:
                formatted = sentence
    if sentence == "":
        return(" ")
    formattedSentence = formatted.split()
    for i in range(len(formattedSentence)):#goes through all the words in the list
            if formattedSentence[i] not in stops:#if the word is not a stop word
                formattedSentence[i]="[MASK]"
                masked = ' '.join(formattedSentence)
                formattedSentence = formatted.split()
                for j in range(5):
                    changedSentence = unmasker(masked)[j]["sequence"]
                    sentences.append(changedSentence)
    if len(sentences)==0:
        return sentence
    return sentences

In [ ]:
filename = "Adversarial_Word_Replace_PyTorch_VPN.csv"
original = []
with open(filename, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        original.append(row)
attacked = original[2:]

In [ ]:
sentences = []
for i in attacked:
    sentences.append(i[0])

In [ ]:
defensePredictions = []
runtimes=  []
total = 0
counter=0
for i in sentences:
    start = timer()
    changed = replace(i)
    end = timer()
    total+=(end-start)
    runtimes.append(end-start)
    classifications = []
    avgConfidence = 0
    cont = 0
    for a in changed:
        b = classification_task(i)
        classifications.append(b)
    totalSum = 0
    for i in classifications:
        if i[0]["label"]=='POSITIVE':
            totalSum+=i[0]["score"]
        else:
            totalSum-=i[0]["score"]
        cont+=1
    if totalSum>0:
        prediction = "POSITIVE"
    elif totalSum<0:
        prediction = "NEGATIVE"
    else:
        prediction = "NEUTRAL"
    avgConfidence = totalSum/cont
    defensePredictions.append([prediction,avgConfidence])
    counter+=1
    print(counter)

In [ ]:
givenLabels = []
predictedLabels=[]
for i in range(len(sentences)):
    givenLabels.append(batch1[1][i])
    if defensePredictions[i][0]=="POSITIVE":
        predictedLabels.append(1)
    elif defensePredictions[i][0]=="NEGATIVE":
        predictedLabels.append(0)
    else:
        predictedLabels.append("")

In [ ]:
numTruePositives = 0
numTrueNegatives = 0
numFalsePositives = 0
numFalseNegatives = 0

In [ ]:
outputs = []
for i in range(len(predictedLabels)):
    correct = False
    if predictedLabels[i]==givenLabels[i]:
        correct = True
    outputs.append([sentences[i],givenLabels[i],predictedLabels[i],defensePredictions[i][1],correct,runtimes[i]])

In [ ]:
for i in outputs:
    if i[4] and i[2]==0:
        numTrueNegatives+=1
    elif i[4] and i[2]==1:
        numTruePositives+=1
    elif not i[4] and i[2]==1:
        numFalsePositives+=1
    elif not i[4] and i[2]==1:
        numFalseNegatives+=1

In [ ]:
accuracy = (numTruePositives+numTrueNegatives)/(numTruePositives+numTrueNegatives+numFalsePositives+numFalseNegatives)
precision = numTruePositives/(numTruePositives+numFalsePositives)
recall = numTruePositives/(numTruePositives+numFalseNegatives)
specificity =numTrueNegatives/(numTrueNegatives+numFalsePositives)
f1 = 2*(precision*recall)/(precision+recall)
totalTime = sum(runtimes)

In [ ]:
filename = "Defense_Word_Replace_PyTorch_VPN.csv"#NEED TO CHANGE IF DIFF DATASET
col_names = ["Sentence", "Actual Label","Predicted Label","Confidence","Runtime"]
row1 = [f"Accuracy:{accuracy:.3f}",f"Precision:{precision:.3f}",f"Recall:{recall:.3f}",f"Specificity:{specificity:.3f}",f"F1-Score:{f1:.3f}",f"Total Runtime:{totalTime:.3f}"]
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
     
    # writing the fields
    csvwriter.writerow(row1)
    csvwriter.writerow(col_names)
     
    # writing the data rows
    csvwriter.writerows(outputs)